In [1]:
import sys
sys.path.append("../modules")
from eda import EDAAnalyzer
from spark_session import SparkManager
from feature_engineering import FeatureEngineer
from ml_developer import XGBoostModelBuilder

In [2]:
# Load autoreload extension
%load_ext autoreload
%autoreload 2

#### Constants and config

In [3]:
mx_submits_path = "../data_sample/mx_submits.parquet/"
mx_submits_line_path = "../data_sample/mx_submitsline.parquet/"


sample_patient_id = "8aad41f612a7095449888c8050abaeb05fdee65643caa3033542610421d8bd1daaa2c4ce1757401003a1bbcd60948a7aa13eba507a676dea80e0cf76b77dbc95"
features_cols = ['secondary_payer_state',
'billing_provider_address_precision',
'billing_provider_address_region',
'claim_filing_indicator_pay_type',
'claim_institutional_or_professional',
'facility_provider_address_precision',
'facility_provider_address_region',
'inpatient_discharge_status_code',
'organization_npi_type_code',
'organization_sourced_from',
'organization_taxonomy_group',
'patient_gender',
'patient_location_residential_region',
'primary_payer_pay_type',
'primary_payer_plan_type',
'principal_diagnosis_body_part',
'principal_diagnosis_category',
'principal_diagnosis_code_set',
'principal_procedure_code_set',
'referring_provider_taxonomy_group',
'rendering_provider_npi_type_code',
'rendering_provider_taxonomy_group',
'secondary_payer_claim_filing_indicator_code',
'secondary_payer_pay_type',
'secondary_payer_plan_type',
'claim_all_diagnosis_codes',
'previous_line_level_procedure_ohe']
exclude_cols = ['patient_id']
label_column = 'claim_total_charge_amount'
most_repeated_diagnosis_list = [] 

In [4]:
mx_submits_spark_manager = SparkManager(mx_submits_path)

24/10/16 22:26:27 WARN Utils: Your hostname, Chaopings-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.0.0.231 instead (on interface en0)
24/10/16 22:26:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/16 22:26:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/10/16 22:26:29 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### Feature Engineering

In [5]:
mx_submits_fe=FeatureEngineer(mx_submits_spark_manager)

In [6]:
mx_submits_fe.add_procedures_with_exponential_decay_sparse_vector()

,previous_line_level_procedure_ohe
0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [7]:
mx_submits_fe.retain_columns(features_cols+[label_column])

In [8]:
mx_submits_fe.convert_columns_to_float(["claim_total_charge_amount"])
preprocess_data = mx_submits_fe.preprocess_procedure_data(exclude_cols=exclude_cols)
preprocess_data

Casted claim_total_charge_amount to float


One-Hot Encoding applied successfully to column: secondary_payer_state
One-Hot Encoding applied successfully to column: billing_provider_address_precision
One-Hot Encoding applied successfully to column: billing_provider_address_region
One-Hot Encoding applied successfully to column: claim_filing_indicator_pay_type
One-Hot Encoding applied successfully to column: claim_institutional_or_professional
One-Hot Encoding applied successfully to column: facility_provider_address_precision
One-Hot Encoding applied successfully to column: facility_provider_address_region
One-Hot Encoding applied successfully to column: inpatient_discharge_status_code
One-Hot Encoding applied successfully to column: organization_npi_type_code
One-Hot Encoding applied successfully to column: organization_sourced_from
One-Hot Encoding applied successfully to column: organization_taxonomy_group
One-Hot Encoding applied successfully to column: patient_gender
One-Hot Encoding applied successfully to column: patient_l

Preprocessing complete. Feature vector created.


In [9]:
model_feature_col = ['line_level_procedure_U0005',
 'line_level_procedure_99291',
 'line_level_procedure_1036F',
 'line_level_procedure_A4657',
 'line_level_procedure_A7034',
 'line_level_procedure_99305',
 'line_level_procedure_G8730',
 'line_level_procedure_85652',
 'line_level_procedure_T1016',
 'line_level_procedure_0W3P8ZZ',
 'line_level_procedure_97150',
 'line_level_procedure_73502',
 'line_level_procedure_Z7610',
 'line_level_procedure_A4239',
 'line_level_procedure_G0153',
 'line_level_procedure_84075',
 'line_level_procedure_71020',
 'line_level_procedure_70486',
 'line_level_procedure_99243',
 'line_level_procedure_G0009',
 'line_level_procedure_92002',
 'line_level_procedure_92136',
 'line_level_procedure_G9903',
 'line_level_procedure_J0696',
 'line_level_procedure_76700',
 'line_level_procedure_G0483',
 'line_level_procedure_1123F',
 'line_level_procedure_92579',
 'line_level_procedure_A9270',
 'line_level_procedure_A4604',
 'line_level_procedure_G0444',
 'line_level_procedure_73590',
 'line_level_procedure_99051',
 'line_level_procedure_84100',
 'line_level_procedure_97762',
 'line_level_procedure_72050',
 'line_level_procedure_93010',
 'line_level_procedure_J1094',
 'line_level_procedure_33208',
 'line_level_procedure_43245',
 'line_level_procedure_99490',
 'line_level_procedure_87880',
 'line_level_procedure_99285',
 'line_level_procedure_1090F',
 'line_level_procedure_J7060',
 'line_level_procedure_G8510',
 'line_level_procedure_83655',
 'line_level_procedure_30233N1',
 'line_level_procedure_J0780',
 'line_level_procedure_88312',
 'line_level_procedure_72070',
 'line_level_procedure_95004',
 'line_level_procedure_90713',
 'line_level_procedure_99222',
 'line_level_procedure_51702',
 'line_level_procedure_99233',
 'line_level_procedure_99173',
 'line_level_procedure_J3010',
 'line_level_procedure_92012',
 'line_level_procedure_83036',
 'line_level_procedure_99394',
 'line_level_procedure_73030',
 'line_level_procedure_89320',
 'line_level_procedure_99396',
 'line_level_procedure_86803',
 'line_level_procedure_G8400',
 'line_level_procedure_27447',
 'line_level_procedure_84295',
 'line_level_procedure_J3420',
 'line_level_procedure_82248',
 'line_level_procedure_1220F',
 'line_level_procedure_0002A',
 'line_level_procedure_99214',
 'line_level_procedure_4040F',
 'line_level_procedure_73560',
 'line_level_procedure_B9998',
 'line_level_procedure_92557',
 'line_level_procedure_84520',
 'line_level_procedure_99349',
 'line_level_procedure_82550',
 'line_level_procedure_G8979',
 'claim_total_charge_amount',
 'line_level_procedure_J1756',
 'line_level_procedure_G8417',
 'line_level_procedure_93978',
 'line_level_procedure_W1793',
 'line_level_procedure_W1760',
 'line_level_procedure_93297',
 'line_level_procedure_G0101',
 'line_level_procedure_87491',
 'line_level_procedure_93308',
 'line_level_procedure_70360',
 'line_level_procedure_G0467',
 'line_level_procedure_86611',
 'line_level_procedure_87507',
 'line_level_procedure_99223',
 'line_level_procedure_73070',
 'line_level_procedure_A0431',
 'line_level_procedure_70450',
 'line_level_procedure_92134',
 'line_level_procedure_99393',
 'line_level_procedure_90460',
 'line_level_procedure_83880',
 'line_level_procedure_G0480',
 'line_level_procedure_U0003',
 'line_level_procedure_92587',
 'line_level_procedure_59425',
 'line_level_procedure_87081',
 'line_level_procedure_99212',
 'line_level_procedure_43212',
 'line_level_procedure_99232',
 'line_level_procedure_83631',
 'line_level_procedure_42820',
 'line_level_procedure_85027',
 'line_level_procedure_12032',
 'line_level_procedure_87563',
 'line_level_procedure_74177',
 'line_level_procedure_00840',
 'line_level_procedure_00731',
 'line_level_procedure_76776',
 'line_level_procedure_86580',
 'line_level_procedure_87186',
 'line_level_procedure_90832',
 'line_level_procedure_99238',
 'line_level_procedure_A7035',
 'line_level_procedure_A4332',
 'line_level_procedure_83921',
 'line_level_procedure_J2060',
 'line_level_procedure_97161',
 'line_level_procedure_H0020',
 'line_level_procedure_77336',
 'line_level_procedure_84703',
 'line_level_procedure_94010',
 'line_level_procedure_H0038',
 'line_level_procedure_82728',
 'line_level_procedure_88311',
 'line_level_procedure_99225',
 'line_level_procedure_87624',
 'line_level_procedure_31231',
 'line_level_procedure_T1019',
 'line_level_procedure_96361',
 'line_level_procedure_82570',
 'line_level_procedure_87086',
 'line_level_procedure_88175',
 'line_level_procedure_97110',
 'line_level_procedure_90732',
 'line_level_procedure_10060',
 'line_level_procedure_80197',
 'line_level_procedure_82378',
 'line_level_procedure_A7030',
 'line_level_procedure_K0738',
 'line_level_procedure_93000',
 'line_level_procedure_A0100',
 'line_level_procedure_E0601',
 'line_level_procedure_99281',
 'line_level_procedure_J1644',
 'line_level_procedure_0HPT0JZ',
 'line_level_procedure_93306',
 'line_level_procedure_J1040',
 'line_level_procedure_J2357',
 'line_level_procedure_J1100',
 'line_level_procedure_91301',
 'line_level_procedure_77080',
 'line_level_procedure_90480',
 'line_level_procedure_00740',
 'line_level_procedure_90686',
 'line_level_procedure_64483',
 'line_level_procedure_05UY07Z',
 'line_level_procedure_S9343',
 'line_level_procedure_72157',
 'line_level_procedure_90837',
 'line_level_procedure_87635',
 'line_level_procedure_88304',
 'line_level_procedure_87430',
 'line_level_procedure_3074F',
 'line_level_procedure_93272',
 'line_level_procedure_A6216',
 'line_level_procedure_J1815',
 'line_level_procedure_82274',
 'line_level_procedure_88309',
 'line_level_procedure_A7038',
 'line_level_procedure_74220',
 'line_level_procedure_45385',
 'line_level_procedure_C9803',
 'line_level_procedure_80048',
 'line_level_procedure_K0003',
 'line_level_procedure_J0690',
 'line_level_procedure_82670',
 'line_level_procedure_93296',
 'line_level_procedure_96372',
 'line_level_procedure_84146',
 'line_level_procedure_94760',
 'line_level_procedure_84443',
 'line_level_procedure_87389',
 'line_level_procedure_H0033',
 'line_level_procedure_87591',
 'line_level_procedure_A7031',
 'line_level_procedure_H0032',
 'line_level_procedure_58300',
 'line_level_procedure_J2270',
 'line_level_procedure_90940',
 'line_level_procedure_90999',
 'line_level_procedure_99070',
 'line_level_procedure_88300',
 'line_level_procedure_87804',
 'line_level_procedure_R0070',
 'line_level_procedure_99213',
 'line_level_procedure_J0330',
 'line_level_procedure_A0436',
 'line_level_procedure_73130',
 'line_level_procedure_92551',
 'line_level_procedure_71260',
 'line_level_procedure_87046',
 'line_level_procedure_95117',
 'line_level_procedure_93295',
 'line_level_procedure_96127',
 'line_level_procedure_0HRV079',
 'line_level_procedure_G0121',
 'line_level_procedure_94618',
 'line_level_procedure_84480',
 'line_level_procedure_86334',
 'line_level_procedure_87505',
 'line_level_procedure_99215',
 'line_level_procedure_45381',
 'line_level_procedure_99457',
 'line_level_procedure_G6015',
 'line_level_procedure_01400',
 'line_level_procedure_97140',
 'line_level_procedure_J3301',
 'line_level_procedure_57454',
 'line_level_procedure_74360',
 'line_level_procedure_86008',
 'line_level_procedure_G8483',
 'line_level_procedure_S0302',
 'line_level_procedure_97035',
 'line_level_procedure_90715',
 'line_level_procedure_90688',
 'line_level_procedure_96365',
 'line_level_procedure_97112',
 'line_level_procedure_3078F',
 'line_level_procedure_00537',
 'line_level_procedure_93005',
 'line_level_procedure_70487',
 'line_level_procedure_3079F',
 'line_level_procedure_99231',
 'line_level_procedure_72100',
 'line_level_procedure_71250',
 'line_level_procedure_82746',
 'line_level_procedure_0241U',
 'line_level_procedure_07B80ZX',
 'line_level_procedure_90471',
 'line_level_procedure_82397',
 'line_level_procedure_G8427',
 'line_level_procedure_97124',
 'line_level_procedure_82607',
 'line_level_procedure_81003',
 'line_level_procedure_99282',
 'line_level_procedure_90833',
 'line_level_procedure_0012A',
 'line_level_procedure_82272',
 'line_level_procedure_E1390',
 'line_level_procedure_99391',
 'line_level_procedure_92550',
 'line_level_procedure_20610',
 'line_level_procedure_82150',
 'line_level_procedure_G2211',
 'line_level_procedure_81001',
 'line_level_procedure_66984',
 'line_level_procedure_71010',
 'line_level_procedure_G0481',
 'line_level_procedure_0WQF0ZZ',
 'line_level_procedure_80053',
 'line_level_procedure_86140',
 'line_level_procedure_E1392',
 'line_level_procedure_87426',
 'line_level_procedure_97153',
 'line_level_procedure_S5161',
 'line_level_procedure_83002',
 'line_level_procedure_97014',
 'line_level_procedure_69210',
 'line_level_procedure_00170',
 'line_level_procedure_93922',
 'line_level_procedure_3017F',
 'line_level_procedure_G8482',
 'line_level_procedure_0HPU0JZ',
 'line_level_procedure_76000',
 'line_level_procedure_20605',
 'line_level_procedure_D0274',
 'line_level_procedure_H2017',
 'line_level_procedure_T1015',
 'line_level_procedure_80307',
 'line_level_procedure_99458',
 'line_level_procedure_82435',
 'line_level_procedure_82105',
 'line_level_procedure_99205',
 'line_level_procedure_86301',
 'line_level_procedure_80069',
 'line_level_procedure_2000F',
 'line_level_procedure_C1876',
 'line_level_procedure_84479']

In [10]:
mx_submits_fe.expand_procedure_features(model_feature_col)

Created column for feature: claim_total_charge_amount
Created column for procedure code: C9803 (index: 0)
Created column for procedure code: 99213 (index: 1)
Created column for procedure code: 87591 (index: 2)
Created column for procedure code: H0033 (index: 3)
Created column for procedure code: J3301 (index: 4)
Created column for procedure code: 93295 (index: 5)
Created column for procedure code: 87804 (index: 6)
Created column for procedure code: 84480 (index: 7)
Created column for procedure code: 92551 (index: 8)
Created column for procedure code: A7031 (index: 9)
Created column for procedure code: 86334 (index: 10)
Created column for procedure code: 96127 (index: 11)
Created column for procedure code: 97035 (index: 12)
Created column for procedure code: G8427 (index: 13)
Created column for procedure code: G0121 (index: 14)
Created column for procedure code: 3079F (index: 15)
Created column for procedure code: 3078F (index: 16)
Created column for procedure code: 96365 (index: 17)
Cr

In [11]:
print(mx_submits_fe.feature_cols)

['secondary_payer_state_ohe', 'billing_provider_address_precision_ohe', 'billing_provider_address_region_ohe', 'claim_filing_indicator_pay_type_ohe', 'claim_institutional_or_professional_ohe', 'facility_provider_address_precision_ohe', 'facility_provider_address_region_ohe', 'inpatient_discharge_status_code_ohe', 'organization_npi_type_code_ohe', 'organization_sourced_from_ohe', 'organization_taxonomy_group_ohe', 'patient_gender_ohe', 'patient_location_residential_region_ohe', 'primary_payer_pay_type_ohe', 'primary_payer_plan_type_ohe', 'principal_diagnosis_body_part_ohe', 'principal_diagnosis_category_ohe', 'principal_diagnosis_code_set_ohe', 'principal_procedure_code_set_ohe', 'referring_provider_taxonomy_group_ohe', 'rendering_provider_npi_type_code_ohe', 'rendering_provider_taxonomy_group_ohe', 'secondary_payer_claim_filing_indicator_code_ohe', 'secondary_payer_pay_type_ohe', 'secondary_payer_plan_type_ohe', 'claim_total_charge_amount', 'previous_line_level_procedure_ohe']


In [12]:
mx_submits_fe.retain_columns(model_feature_col+[label_column])

In [13]:
model_data = mx_submits_fe.preprocess_features(model_feature_col, label_column)
model_data

DataFrame[features: vector, claim_total_charge_amount: float]

## Model Training

In [14]:
xgb_model = XGBoostModelBuilder(model_data, model_feature_col, label_column)

In [15]:
xgb_model.split_data()

(DataFrame[features: vector, claim_total_charge_amount: float],
 DataFrame[features: vector, claim_total_charge_amount: float])

In [ ]:
xgb_model.train_model()

In [ ]:
xgb_model.evaluate_model()

In [ ]:
rmse_train = xgb_model.evaluate_model(type="Train")
print(f"RMSE on training data: {rmse_train}")